In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

### Initializing Model

In [15]:
m = pyo.ConcreteModel()

### Set Parameteres (What is given?)

In [16]:
m.set0 = pyo.Set(initialize=['A', 'B', 'C', 'D', 'E'])  #names of open projects
m.R = {
    'A': 500,
    'B': 4000,
    'C': 3000,
    'D': 2000,
    'E': 2000
}  #revenue by project
m.NT = {
    'A': 1,
    'B': 3,
    'C': 2,
    'D': 1,
    'E': 5
}  #number of teams required by the project
m.Nteams = 5 #total teams avaliable to do the projects

### Variables (What do we need to find)?

In [17]:
m.x = pyo.Var(m.set0,
              within=pyo.Binary)  #which open projects will be picked up

### Objective Function (How do we find the best possible outcome)?

- m.x[project_name] == 0 if the project is not active
- m.x[project_name] == 1 if the project is active

In [18]:
m.obj = pyo.Objective(
    expr=sum([m.x[o] * m.R[o] for o in m.set0]),
    sense=pyo.maximize)  #product of active project * revenue for that projeect

### Constrains (What are the conditions we need to keep in mind)?

In [19]:
m.C1 = pyo.Constraint(expr=sum([m.x[o] * m.NT[o] for o in m.set0]) <=
                      m.Nteams)  #total number of teams should be less than 5
m.C2 = pyo.Constraint(
    expr=m.x['C'] <= m.x['A'])  #project A must happen for C to happen
m.C3 = pyo.Constraint(expr=m.x['D'] <= m.x['A'])
m.C4 = pyo.Constraint(
    expr=m.x['D'] <= m.x['C'])  # project A and C must happen for D to happen

### Solveing using Solver

In [20]:
#solve
opt = SolverFactory('gurobi')
m.results = opt.solve(m)

### Results

In [21]:
m.pprint()
print('\n\nOF:',pyo.value(m.obj))

1 Set Declarations
    set0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'A', 'B', 'C', 'D', 'E'}

1 Var Declarations
    x : Size=5, Index=set0
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :   1.0 :     1 : False : False : Binary
          B :     0 :   0.0 :     1 : False : False : Binary
          C :     0 :   1.0 :     1 : False : False : Binary
          D :     0 :   1.0 :     1 : False : False : Binary
          E :     0 :   0.0 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 500*x[A] + 4000*x[B] + 3000*x[C] + 2000*x[D] + 2000*x[E]

4 Constraint Declarations
    C1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                                   : Upper : Active
        None :  -Inf : x[A] + 3*x[B] + 2*x[C] + x[D] +